Load API Keys

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

Load Contents from Brillar Bank

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from helpers.test_data import get_urls

loader = WebBaseLoader(get_urls())

data = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.
d:\Code\langchain_python\virtualenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Split the Loaded Data into Chunks

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=20)
texts = text_splitter.split_documents(data)

Get Embedding Model

In [4]:
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=256)

Instantiate Elasticsearch Store

In [5]:
from langchain_elasticsearch import ElasticsearchStore, DenseVectorStrategy, BM25Strategy
import os

es_cloud_id = os.getenv("ELASTIC_CLOUD_ID")
es_api_key = os.getenv("ELASTIC_API_KEY")


vector_store = ElasticsearchStore.from_documents(
    documents=texts,
    es_cloud_id=es_cloud_id,
    es_api_key=es_api_key,
    index_name="hybrid_search1",
    embedding=embedding,
    strategy=DenseVectorStrategy(hybrid=True),
)


Get LLM

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

Get Retriever

In [7]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

Create Chain

In [8]:
from helpers.conversation_retrieval_chain import create_conversational_retrieval_chain

chain = create_conversational_retrieval_chain(llm, retriever)

Invoke Chain

In [9]:
from helpers.conversation_retrieval_chain import invoke_conversational_retrieval_chain
from helpers.test_data import get_questions

questions = get_questions()
for question in questions:
  result = invoke_conversational_retrieval_chain(chain, question)
  print(f'{question}\n{result["answer"]}\n')

Tell me what is Brillar bank?
Brillar Bank is a financial institution dedicated to helping customers achieve financial success through savings, investments, and expert advice. It offers various types of fixed deposit accounts, including Flexi Fixed Deposit, e-Fixed Deposit, and Foreign Currency Fixed Deposit, with attractive interest rates and flexible terms. Brillar Bank is a member of PIDM and ensures protection for depositors up to RM250,000.

How many types of fixed deposit does Brillar Bank provide?
Brillar Bank provides six types of fixed deposits: Fixed Deposit, Senior Savers Flexi Fixed Deposit, e-Fixed Deposit, Junior Fixed Deposit, Flexi Fixed Deposit, and Foreign Currency Fixed Deposit.

What are the interest rates for fixed deposit?
The interest rates for fixed deposits at Brillar Bank are as follows:

- 1 month: 2.15%
- 2 - 3 months: 2.25%
- 4 - 5 months: 2.30%
- 6 months: 2.30%
- 7 - 11 months: 2.35%
- 12 - 60 months: 2.35%

What are the interest rates for e-fixed deposit